In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
AI_STUDIO_KEY = os.getenv("GOOGLE_AI_STUDIO_KEY")
print(AI_STUDIO_KEY)

## Connect to Weaviate

In [ ]:
import weaviate, os, json

client = weaviate.connect_to_wcs(
    cluster_url=os.getenv("WORKSHOP_DEMO_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WORKSHOP_DEMO_KEY_READ")),
    
    headers={
       "X-Palm-Api-Key": AI_STUDIO_KEY,
    }
)

client.is_ready()


In [ ]:
import weaviate, os, json

# Connect to your local Weaviate instance
client = weaviate.connect_to_local(
    headers={
       "X-Google-Api-Key": AI_STUDIO_KEY,
    }
)

client.is_ready()

## Search

The `alpha` parameter determines the weight given to the sparse and dense search methods. `alpha = 0` is pure sparse (bm25) search, whereas `alpha = 1` is pure dense (vector) search. 

Alpha is an optional parameter. The default is set to `0.75`.

### Semantic Search

In [ ]:
products = client.collections.get("Products")

response = products.query.near_text(
    # query="yoga top",
    # query="running shoes",
    # query="buty do biegania dla kobiet",
    query="trainers for girls",
    limit=5
)

for item in response.objects:
    print(item.properties["name"])
    # print(item.properties["sub_category"], "\n")
    # print(json.dumps(item.properties, indent=2))

### Keyword Search

In [ ]:
import weaviate.classes.query as wq
products = client.collections.get("Products")

response = products.query.bm25(
    # query="running shoes",
    query="trainers for girls",
    query_properties=["name"],
    limit=5
)

for item in response.objects:
    print(item.properties["name"])

### Hybrid Search

In [ ]:
products = client.collections.get("Products")

response = products.query.hybrid(
    # query="yoga top",
    # query="running shoes",
    # query="running shoes for girls",
    query="trainers for girls",
    alpha=0.5,
    query_properties=["name"],
    limit=5,
)

for item in response.objects:
    print(item.properties["name"])

### Explain score

In [ ]:
products = client.collections.get("Products")

response = products.query.hybrid(
    query="trainers for girls",
    alpha=0.5,
    query_properties=["name"],
    limit=5,
    return_metadata=wq.MetadataQuery(score=True, explain_score=True)
)

for item in response.objects:
    print(item.properties["name"])

    print(item.metadata.explain_score)
    print(item.metadata.score)
    print("=========")


### Fusion Type

In [ ]:
import weaviate.classes.query as wq
response = products.query.hybrid(
    query="trainers for girls",
    query_properties=["name", "sub_category"],
    alpha=0.5,
    limit=5,
    fusion_type=wq.HybridFusion.RELATIVE_SCORE,
    return_metadata=wq.MetadataQuery(score=True, explain_score=True)
)

for item in response.objects:
    print(item.properties["name"])

    print(item.metadata.explain_score)
    print(item.metadata.score)
    print("=========")

In [ ]:
import weaviate.classes.query as wq
response = products.query.hybrid(
    query="trainers for girls",
    query_properties=["name", "sub_category"],
    alpha=0.5,
    limit=5,
    fusion_type=wq.HybridFusion.RANKED,
    return_metadata=wq.MetadataQuery(score=True, explain_score=True)
)

for item in response.objects:
    print(item.properties["name"])

    print(item.metadata.explain_score)
    print(item.metadata.score)
    print("=========")

### Hybrid Search with a `where` filter

Find Jeopardy questions about elephants, where the category is set to Animals.

In [ ]:
import weaviate.classes.query as wq

response = products.query.hybrid(
    query="running trousers",
    alpha=0.7,
    filters=wq.Filter.by_property("ratings").greater_than(4),
    limit=3
)

for item in response.objects:
    print(item.properties["name"])
    print(item.properties["ratings"])

## Generative Search

In [ ]:
response = products.generate.hybrid(
    query="running trousers",
    alpha=0.5,
    filters=wq.Filter.by_property("ratings").greater_than(4),
    limit=3,
    single_prompt="Please give me a concise name for the following product, together with a one-paragraph description. {name}"
)

for item in response.objects:
    print("Original name:", item.properties["name"])
    print(item.generated)
    print("=======\n")

In [ ]:
response = products.generate.hybrid(
    query="running trousers",
    alpha=0.5,
    filters=wq.Filter.by_property("ratings").greater_than(4),
    limit=3,
    grouped_task="What do the provided products have in common? Please write a short advert I could post on LinkedIn.",
    grouped_properties=["name"]
)

print(response.generated)